In [4]:
# UART to Avalon-MM bridge送受信

from serial import *
from struct import *

def print_bin(label, bin):
	print(label, end=" : ")
	for b in bin: print(format(b, "02x"), end=" ")
	print("")


# Packetデータのデコードとエンコード

def bytes_decode(byte):
	if byte == 0x7a:
		bytes_decode.status |= 1<<0
		bytes_decode.result = b""
		return
	elif byte == 0x7b:
		bytes_decode.status |= 1<<1
		return
	elif byte == 0x7c:
		bytes_decode.status |= 1<<2
		return
	elif byte == 0x7d:
		bytes_decode.status |= 1<<3
		return

	if (bytes_decode.status & (1<<2)) != 0:
		bytes_decode.status &= ~(1<<2)
		return

	if (bytes_decode.status & (1<<3)) != 0:
		bytes_decode.status &= ~(1<<3)
		byte ^= 0x20

	if (bytes_decode.status & (1<<0)) != 0:
		bytes_decode.result += pack("B", byte)

	if (bytes_decode.status & (1<<1)) != 0:
		bytes_decode.status = 0
		return bytes_decode.result

bytes_decode.status = 0
bytes_decode.result = b""

def bytes_encode(bin):
	bytes = b"\x7c\x00\x7a"
	for idx,c in enumerate(bin):
		if idx == len(bin) - 1:
			bytes += b"\x7b"

		if c >= 0x7a and c <= 0x7d:
			bytes += pack("BB", 0x7d, c ^ 0x20)
		else:
			bytes += pack("B", c)
	return bytes


# Avalon-MMのライトとリード

def avm_memwr(ser, addr, wrbytes):
	send_data = bytes_encode(pack("!BxHL", 0x04, len(wrbytes), addr) + wrbytes)
	print_bin("-> MEMWR", send_data)
	ser.write(send_data)
	res = None
	while res is None: res = bytes_decode(ser.read()[0])
	print_bin("<-   RES", res)
	resp, size = unpack("!BxH", res[0:4])
	return size

def avm_memrd(ser, addr, readnum):
	send_data = bytes_encode(pack("!BxHL", 0x14, readnum, addr))
	print_bin("-> MEMRD", send_data)
	ser.write(send_data)
	res = None
	while res is None: res = bytes_decode(ser.read()[0])
	print_bin("<-   RES", res)
	return res


In [5]:
# Platform Designerモジュールのアクセス 

from datetime import *

SERIAL_PORT = "COM5"
SERIAL_BITRATE = 115200

with Serial(SERIAL_PORT, SERIAL_BITRATE) as ser:
	ser.reset_input_buffer()

	# SystemIDペリフェラルのリード

	rdbytes = avm_memrd(ser, 0x10000000, 8)
	sysid, timestamp = unpack('LL', rdbytes[0:8])
	dt = datetime.fromtimestamp(timestamp)
	print("System ID =", hex(sysid), ", Build", dt)

	# PIOペリフェラルのライト（LED点灯）

	avm_memwr(ser, 0x10000020, b"\x01\x00\x00\x00")
	print("UserLED[0] on")


-> MEMRD : 7c 00 7a 14 00 00 08 10 00 00 7b 00 
<-   RES : 01 00 a0 72 47 99 87 63 
System ID = 0x72a00001 , Build 2022-12-01 02:56:23
-> MEMWR : 7c 00 7a 04 00 00 04 10 00 00 20 01 00 00 7b 00 
<-   RES : 84 00 00 04 
UserLED[0] on
